In [193]:
puzzle = open("AoC16.txt").read().splitlines()
puzzle = [i.split(" ") for i in puzzle]
valves = [i[1] for i in puzzle]
flowrates = [int(i[4].split("=")[-1][:-1]) for i in puzzle]
outputs = {}
for i in range(len(puzzle)):
    ans = []
    for j in range(9, len(puzzle[i])):
        ans.append(puzzle[i][j][:2])
    outputs[valves[i]]=ans
flows = {}
for i in range(len(valves)):
    flows[valves[i]]=flowrates[i]

In [194]:
dist = {}
for i in range(len(valves)):
    if flowrates[i]!=0:
        dist[valves[i]]={}
        for j in range(len(valves)):
            if flowrates[j]!=0:
                dist[valves[i]][valves[j]]=abs(i-j)

In [195]:
nonzero = [valves[i] for i in range(len(valves)) if flowrates[i]!=0]
bin = {}
for i in range(len(nonzero)):
    bin[nonzero[i]]=2**i
print(bin)
def inbin(valve, bi):
    return bi & (2**nonzero.index(valve))!=0
def conv(visit):
    ans = []
    for i in range(len(nonzero)):
        if visit & (2**i)!=0:
            ans.append(nonzero[i])
    ans2 = 0
    for i in ans:
        ans2+=flows[i]
    return ans2
print(conv(1))

{'QE': 1, 'TN': 2, 'HK': 4, 'SK': 8, 'BY': 16, 'FE': 32, 'BM': 64, 'EI': 128, 'QN': 256, 'CJ': 512, 'CQ': 1024, 'SZ': 2048, 'XH': 4096, 'KI': 8192, 'NS': 16384}
3


In [196]:
#find the minimum path between two rooms
def findpath(start, end):
    q = [(start, 0)]
    visited = set()
    while q:
        node, dist = q.pop(0)
        if node == end:
            return dist
        if node not in visited:
            visited.add(node)
            for i in outputs[node]:
                q.append((i, dist+1))
    return -1
adjMatrix = {}
for i in nonzero+["AA"]:
    for j in nonzero+["AA"]:
        #if(i!="AA"):
         #   adjMatrix[(i,j)]=findpath(i, j)+1
        #else:
        adjMatrix[(i,j)]=findpath(i, j)
        if(adjMatrix[(i,j)]>=26):
            print(i, j, adjMatrix[i, j])

In [197]:
#find all paths starting from AA that have a total distance of at most 30
#Only go to nodes that have a flowrate of at least 1
#Distance between two nodes i and j is given by adjMatrix[(i,j)]
#Don't visit a node more than once
#The total distance of a path is the sum of the distances between each node
#output a list of paths where each path has the form [(node1, time_left_1), (node2, time_left_2), ...]
#where time_left_i is the amount of time left after visiting node_i
#the first node in the path is always AA
@lru_cache(maxsize=None)
def findpaths(start, time, visited=frozenset()):
    ans = []
    if time <= 0:
        return ans
    if(flows[start]!=0 and start not in visited):
        ans.append([(start, time-1)])
    for i in nonzero:
        if(i not in visited and i != start):
            #if adjMatrix[(start, i)]<=time:
                for j in findpaths(i, time-adjMatrix[(start, i)]-1, visited.union({start})):
                    if(start in visited):
                        print(visited, i)
                    ans.append([(start, time-1)]+j)
    return ans
paths = findpaths("AA", 31)

In [198]:
def findPressure(path):
    ans = 0
    for i in path:
        ans+=flows[i[0]]*i[1]
    return ans
ans = 0
for i in paths:
    ans = max(ans, findPressure(i))
print(ans)

2077


In [199]:
paths2 = findpaths("AA", 27)
print(len(paths2))

62834


In [202]:
def conv(path):
    ans = 0
    for i in range(1,len(path)):
        ans+=bin[path[i][0]]
    return ans
pressures = [findPressure(i) for i in paths2]
bin = [conv(i) for i in paths2]

In [205]:
maxAns2 = 0
for i in range(len(paths2)):
    if(i%1000==0):
        print(i)
    for j in range(i+1, len(paths2)):
        if(bin[i] & bin[j]==0):
            maxAns2 = max(maxAns2, pressures[i]+pressures[j])
print(maxAns2)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
2741


In [ ]:
#Old code for part 1, runs too long

cache = {}
from functools import lru_cache
@lru_cache(maxsize=None)
def dfs(start, time, visited, tot_pressure=0):
    if time == 1:
        return (tot_pressure, visited)
    withValve = (0,0)
    if(flows[start]!=0 and (not inbin(start, visited))):
        c = dfs(start, time-1, visited+bin[start], tot_pressure+conv(visited+bin[start]))
        withValve = c
        return withValve
    withoutValve = (0,0)
    for i in outputs[start]:
        c = dfs(i, time-1, visited, tot_pressure+conv(visited))
        withoutValve = max(withoutValve, c, key=lambda x: x[0])
    winner = (0,0)
    if(withValve[0]<withoutValve[0]):
        winner = withoutValve
    else:
        winner = withValve
    return winner

print(dfs("AA", 30, 0))